In [22]:
from dotenv import load_dotenv
import os
import numpy as np
import openai
from openai import OpenAI
load_dotenv()
apikey = os.getenv("clave")

In [23]:
import faiss
import json

# Cargar el índice FAISS
index = faiss.read_index("faiss_index.index")

# Cargar los documentos originales
with open("embeddings/document_texts.json", "r") as f:
    documentos = json.load(f)

In [28]:
openai.api_key = apikey
def get_embedding(text):
    # Este es el método correcto para la versión >= 1.0.0
    response = openai.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response.data[0].embedding

def search(query_embedding, k=5):
    # Buscar los k documentos más cercanos al embedding de la consulta
    D, I = index.search(np.array([query_embedding]).astype('float32'), k)
    return I, D

def retrieve_context(query_text, k=5):
    # Obtener el embedding de la consulta
    query_embedding = get_embedding(query_text)
    
    # Buscar los documentos más relevantes
    indices, distances = search(query_embedding, k)
    
    # Recuperar los textos correspondientes (si el índice está en rango)
    relevant_texts = [documentos[i] for i in indices[0] if i < len(documentos)]
    
    return " ".join(relevant_texts)

### Modelo fine-tuning con RAG

In [55]:
client = OpenAI(api_key=apikey)
def generate_response(prompt):
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::AwCV0n9Y",  # Ajusta con el nombre de tu modelo fine-tuned
        messages=[
            {"role": "system", "content": "Eres una persona que está teniendo nua conversación distendida con un divulgador de ciencia"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.9,  # Ajusta la creatividad de la respuesta (0.0 - 1.0)
        max_tokens=1150,   # Número máximo de tokens en la respuesta
        top_p=1,          # Control de muestreo por núcleo (0.0 - 1.0)
        frequency_penalty=0,  # Penaliza repeticiones
        presence_penalty=0
    )
    return response.choices[0].message.content

In [56]:
def ask_model(query):
    # Recupera contexto con RAG
    context = retrieve_context(query)
    
    # Crea el prompt combinando el contexto con la consulta
    prompt = f"Contexto relevante: {context}\nPregunta del usuario: {query}"
    
    # Genera la respuesta del modelo
    response = generate_response(prompt)
    
    return response

In [68]:
from rich.console import Console
from rich.markdown import Markdown
query = "Qué descubrió Colón sobre la estrella POlar durante su expedición? Qué le marcaba su brújula?"
response = ask_model(query)
console = Console()
console.print(Markdown(response))

Bueno, resulta que Colón no era físico ni astrónomo... Digamos que Colón estaba dándole vueltas a la brújula sin   
saber muy bien qué pasaba. Para él, todo este rollo era un misterio bastante enmarañado. Veía cómo su brújula      
apuntaba de un lado de la estrella polar al otro conforme avanzaba la noche, y no tenía ni idea de lo que estaba   
sucediendo. Según su diario de a bordo, al comienzo de la noche, las agujas noroesteaban y a la mañana             
nordesteaban. La brújula, en vez de señalar directamente al norte, se iba un poquito al oeste al principio de la   
noche y luego un poquito al este justo antes del amanecer.inceton, en Nueva Jersey, es de unos 12 grados al oeste. 
Vas comprendiendo, ¿no?